**Export the country club data from PHPMyAdmin, and connect to a local SQLite instance 
for the following questions.**

In [1]:
import sqlite3 as sql
import pandas as pd
pd.set_option("max_rows", None) # Allow to display all rows in the cell

con = sql.connect('database/sqlite_db_pythonsqlite.db') # Connect to the db
curs = con.cursor() # Create cursor

sample_query = 'SELECT * FROM Facilities;'
curs.execute(sample_query) # Execute a sample query
rs = curs.fetchall() # Fetch all rows

df = pd.DataFrame(rs) # Copy to a dataframe
df.columns = [tuple[0] for tuple in curs.description] # Asign columns to dataframe
df # Show dataframe

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue.

In [2]:
query10 = ('SELECT subquery.name, subquery.total_revenue '
           'FROM (SELECT Facilities.name, memid, '
                        'SUM(CASE WHEN memid = 0 THEN (guestcost * slots) '
                             'WHEN  memid <> 0 THEN (membercost * slots) '
                             'END) AS total_revenue '
                  'FROM Bookings '
                  'LEFT JOIN Facilities '
                    'USING(facid) '
                  'GROUP BY Facilities.name '
                  ') AS subquery '
            
            'WHERE total_revenue < 1000 ' 
            'ORDER BY total_revenue;'
          )

curs.execute(query10) # Execute a sample query
rs10 = curs.fetchall() # Fetch all rows

df10 = pd.DataFrame(rs10) # Copy to a dataframe
df10.columns = [tuple[0] for tuple in curs.description] # Asign columns to dataframe
df10 # Show dataframe

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [3]:
query11 = ('SELECT m.surname, m.firstname, r.firstname || " " || r.surname AS recommended_by '
           'FROM Members AS m '
           'LEFT JOIN Members AS r '
                 'ON m.recommendedby = r.memid '
           'WHERE m.memid <> 0 ' 
           'ORDER BY m.surname, m.firstname;'
          )

curs.execute(query11) # Execute a sample query
rs11 = curs.fetchall() # Fetch all rows

df11 = pd.DataFrame(rs11) # Copy to a dataframe
df11.columns = [tuple[0] for tuple in curs.description] # Asign columns to dataframe
df11 # Show dataframe

,surname,firstname,recommended_by
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Farrell,David,None
9,Farrell,Jemima,None


Q12: Find the facilities with their usage by member, but not guests

In [4]:
query12 = ('SELECT f.name, m.firstname || " " || m.surname AS member, SUM(b.slots)/2.0 AS usage_time_h '
           'FROM Bookings AS b '
           'LEFT JOIN Members AS m '
                 'USING (memid) '
           'LEFT JOIN Facilities AS f '
                 'USING (facid) '
           'WHERE b.memid <> 0 ' 
           'GROUP BY name, member;'
          )

curs.execute(query12) # Execute a sample query
rs12 = curs.fetchall() # Fetch all rows

df12 = pd.DataFrame(rs12) # Copy to a dataframe
df12.columns = [tuple[0] for tuple in curs.description] # Asign columns to dataframe
df12 # Show dataframe

,name,member,usage_time_h
0,Badminton Court,Anna Mackenzie,48.0
1,Badminton Court,Anne Baker,15.0
2,Badminton Court,Burton Tracy,3.0
3,Badminton Court,Charles Owen,9.0
4,Badminton Court,Darren Smith,216.0
5,Badminton Court,David Jones,12.0
6,Badminton Court,David Pinker,10.5
7,Badminton Court,Douglas Jones,3.0
8,Badminton Court,Erica Crumpet,3.0
9,Badminton Court,Florence Bader,13.5


Q13: Find the facilities usage by month, but not guests

In [5]:
query13 = ('SELECT f.name, strftime("%m", b.starttime) AS month, SUM(b.slots)/2.0 AS usage_time_h '
           'FROM Bookings AS b '
           'LEFT JOIN Facilities AS f '
                 'USING (facid) '
           'WHERE b.memid <> 0 ' 
           'GROUP BY name, month '
           'ORDER BY name, month;'
          )

curs.execute(query13) # Execute a sample query
rs13 = curs.fetchall() # Fetch all rows

df13 = pd.DataFrame(rs13) # Copy to a dataframe
df13.columns = [tuple[0] for tuple in curs.description] # Asign columns to dataframe
df13 # Show dataframe

,name,month,usage_time_h
0,Badminton Court,07,82.5
1,Badminton Court,08,207.0
2,Badminton Court,09,253.5
3,Massage Room 1,07,83.0
4,Massage Room 1,08,158.0
5,Massage Room 1,09,201.0
6,Massage Room 2,07,4.0
7,Massage Room 2,08,9.0
8,Massage Room 2,09,14.0
9,Pool Table,07,55.0


In [6]:
con.close() # Close the connection